# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [4]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [24]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("/Users/laceymorgan/Desktop/Python_API_Challenge/output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,punta arenas,-53.1500,-70.9167,12.06,58,75,12.86,CL,1675177118
1,1,fort nelson,58.8053,-122.7002,-14.23,90,100,3.60,CA,1675177119
2,2,kattivakkam,13.2167,80.3167,27.05,79,100,5.69,IN,1675177119
3,3,hofn,64.2539,-15.2082,1.18,70,100,11.15,IS,1675177119
4,4,busselton,-33.6500,115.3333,16.26,70,0,8.83,AU,1675177120


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [6]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    'Lng','Lat', 
    geo=True, 
    tiles= 'OSM',
    frame_width= 700,
    frame_height= 500,
    color= 'City',
    size= 'Humidity',
)


# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [7]:
# Narrow down cities that fit criteria and drop any results with null values
# Max temperature lower than 27 degrees but higher than 21, wind speed less than 4.5 m/s, zero cloudiness
narrow_df =  city_data_df.loc[(city_data_df['Max Temp']<27) & (city_data_df['Max Temp']>21) &
(city_data_df['Wind Speed']<4.5) & (city_data_df['Cloudiness'] == 0)]

# Drop any rows with null values
narrow_df = narrow_df.dropna()

# Display sample data
narrow_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
223,223,puerto escondido,15.8500,-97.0667,24.46,88,0,2.57,MX,1675177242
268,268,abu dhabi,24.4667,54.3667,26.06,36,0,3.09,AE,1675177031
323,323,champerico,14.3000,-91.9167,24.30,55,0,1.38,GT,1675177308
395,395,punta gorda,26.9298,-82.0454,25.00,84,0,0.45,US,1675177355
428,428,acapulco,16.8634,-99.8901,24.90,78,0,3.09,MX,1675177370


### Step 3: Create a new DataFrame called `hotel_df`.

In [8]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = narrow_df[['City','Country','Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name']=""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
223,puerto escondido,MX,15.8500,-97.0667,88,
268,abu dhabi,AE,24.4667,54.3667,36,
323,champerico,GT,14.3000,-91.9167,55,
395,punta gorda,US,26.9298,-82.0454,84,
428,acapulco,MX,16.8634,-99.8901,78,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [9]:
# Set parameters to search for a hotel

categories = "accommodation.hotel"
radius = 10000
limit = 20

params = {
    "categories":categories,
    "limit":limit,
    "apiKey":geoapify_key 
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
puerto escondido - nearest hotel: Hotel Barlovento
abu dhabi - nearest hotel: Marriott Downtown Abu-Dhabi
champerico - nearest hotel: Hotel y Restaurante El Submarino
punta gorda - nearest hotel: SpringHill Suites by Marriott Punta Gorda Harborside
acapulco - nearest hotel: Hotel del Valle


,City,Country,Lat,Lng,Humidity,Hotel Name
223,puerto escondido,MX,15.8500,-97.0667,88,Hotel Barlovento
268,abu dhabi,AE,24.4667,54.3667,36,Marriott Downtown Abu-Dhabi
323,champerico,GT,14.3000,-91.9167,55,Hotel y Restaurante El Submarino
395,punta gorda,US,26.9298,-82.0454,84,SpringHill Suites by Marriott Punta Gorda Harb...
428,acapulco,MX,16.8634,-99.8901,78,Hotel del Valle


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [21]:
%%capture --no-display

# Configure the map plot
# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    'Lng','Lat', 
    geo=True, 
    tiles= 'OSM',
    frame_width= 700,
    frame_height= 500,
    color= 'City',
    hover_cols=['Hotel Name','Country']
)

# Display the map
map_plot_2


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)